In [11]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Prannaya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Prannaya\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Prannaya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Prannaya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


True

In [8]:
import pandas as pd
import random
import nltk

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

import nlpaug.augmenter.word as word_aug
import nlpaug.augmenter.sentence as sentence_aug
import nlpaug.flow as aug_flow

In [9]:
category_codes = {0: 'Claim',
 1: 'Concluding Statement',
 2: 'Counterclaim',
 3: 'Evidence',
 4: 'Lead',
 5: 'Position',
 6: 'Rebuttal'}
labels = list(zip(*category_codes.items()))[1]
labels

('Claim',
 'Concluding Statement',
 'Counterclaim',
 'Evidence',
 'Lead',
 'Position',
 'Rebuttal')

In [10]:
i = 0
def augment(sentence: str) -> str:
    global i
    print(i, end=" ")
    i += 1
    outputs = []
    for syn in (
          None,
          word_aug.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute"),
          word_aug.ContextualWordEmbsAug(model_path='distilbert-base-uncased', action="substitute"),
          word_aug.ContextualWordEmbsAug(model_path='roberta-base', action="substitute"),
          word_aug.SynonymAug(aug_src='wordnet')
        ):
        for sen in (
        None,
        sentence_aug.ContextualWordEmbsForSentenceAug(model_path='xlnet-base-cased'),
        sentence_aug.ContextualWordEmbsForSentenceAug(model_path='gpt2'),
        sentence_aug.ContextualWordEmbsForSentenceAug(model_path='distilgpt2'),
        sentence_aug.AbstSummAug(model_path='t5-base'),
        ):
            flow = aug_flow.Sequential()
            #print(syn, sen)
            if syn: flow.append(syn)
            if sen: flow.append(sen)
            outputs.append(flow.augment(sentence))
    return outputs

augment("While some may say Artificial Intelligence is akin to these other inventions, I beg to disagree.")

0 

['While some may say Artificial Intelligence is akin to these other inventions, I beg to disagree.',
 'While some may say Artificial Intelligence is akin to these other inventions, I beg to disagree. The concept is essentially a simple operating',
 'While some may say Artificial Intelligence is akin to these other inventions, I beg to disagree. that the being a a that he at an he it .',
 'While some may say Artificial Intelligence is akin to these other inventions, I beg to disagree. B I A This a .',
 "Artificial Intelligence is akin to other inventions, says bob greene. he says some may say it's similar to these inventions. greene: i beg to disagree.",
 'though some fans say artificial hearing is fundamental to these electronic inventions, some beg to disagree.',
 'while some thinkers contend artificial light is related to these other phenomena, i like to disagree. <eop> This would like me',
 "while some may argue artificial intelligence has something too these other concepts, i beg y

In [13]:
df = pd.read_csv("pseudo/pseudolab.csv")
df

,sentence,Claim,Concluding Statement,Counterclaim,Evidence,Lead,Position,Rebuttal,text
0,In 2019 a wave of anti-abortion laws swept thi...,6.635360,-2.915717,-0.175175,1.890639,-2.265393,-2.164053,-0.640354,In 2019 a wave of anti-abortion laws swept thi...
1,But these grabbed the public’s attention in a ...,-0.422903,-1.966259,0.220908,0.146967,-3.457461,-0.901608,6.126251,In 2019 a wave of anti-abortion laws swept thi...
2,Georgia banned abortion after about six weeks ...,1.859675,-3.230959,-0.389729,6.427304,-1.808647,-3.455705,-0.482240,In 2019 a wave of anti-abortion laws swept thi...
3,"Ohio, Mississippi, Louisiana and Kentucky did ...",0.327022,-2.367975,-1.594195,7.579478,-1.537600,-3.283216,-0.565419,In 2019 a wave of anti-abortion laws swept thi...
4,"Alabama went the furthest, banning virtually a...",6.683917,-2.563557,-0.671641,1.514271,-2.902210,-1.887146,-0.006093,In 2019 a wave of anti-abortion laws swept thi...
...,...,...,...,...,...,...,...,...,...
28359,"Jim Justice of West Virginia, a Republican, sa...",-0.826471,-2.144290,-0.996875,8.095543,-1.530808,-3.267610,-0.607475,WASHINGTON — The Biden administration has claw...
28360,The Biden administration had hoped to avoid sh...,3.889133,-3.152361,1.242280,2.615246,-2.932186,-2.632563,0.941492,WASHINGTON — The Biden administration has claw...
28361,"Late last year, the White House persuaded Ariz...",5.278596,-2.919935,-0.322822,2.819602,-2.090031,-2.463582,-0.311356,WASHINGTON — The Biden administration has claw...
28362,Yet administration officials are less concerne...,-0.367214,-2.063666,0.582598,0.422793,-3.704168,-1.220427,6.007252,WASHINGTON — The Biden administration has claw...


In [14]:
from torch.nn.functional import softmax
import torch

def softermax(row):
    return pd.Series([float(i) for i in softmax(torch.tensor(row.tolist()), dim=-1)])

probs = df[list(labels)].apply(softermax, 1)
probs.columns = labels
stdval = probs.std(axis=1)
df_fil = df.iloc[stdval[stdval >= stdval.sort_values().iloc[5771:5773].mean()].index]
df_fil

,sentence,Claim,Concluding Statement,Counterclaim,Evidence,Lead,Position,Rebuttal,text
0,In 2019 a wave of anti-abortion laws swept thi...,6.635360,-2.915717,-0.175175,1.890639,-2.265393,-2.164053,-0.640354,In 2019 a wave of anti-abortion laws swept thi...
1,But these grabbed the public’s attention in a ...,-0.422903,-1.966259,0.220908,0.146967,-3.457461,-0.901608,6.126251,In 2019 a wave of anti-abortion laws swept thi...
2,Georgia banned abortion after about six weeks ...,1.859675,-3.230959,-0.389729,6.427304,-1.808647,-3.455705,-0.482240,In 2019 a wave of anti-abortion laws swept thi...
3,"Ohio, Mississippi, Louisiana and Kentucky did ...",0.327022,-2.367975,-1.594195,7.579478,-1.537600,-3.283216,-0.565419,In 2019 a wave of anti-abortion laws swept thi...
4,"Alabama went the furthest, banning virtually a...",6.683917,-2.563557,-0.671641,1.514271,-2.902210,-1.887146,-0.006093,In 2019 a wave of anti-abortion laws swept thi...
...,...,...,...,...,...,...,...,...,...
28358,A spokesman for Gov.,7.302837,-2.645599,-0.514667,0.703273,-2.210048,-1.439218,-0.953753,WASHINGTON — The Biden administration has claw...
28359,"Jim Justice of West Virginia, a Republican, sa...",-0.826471,-2.144290,-0.996875,8.095543,-1.530808,-3.267610,-0.607475,WASHINGTON — The Biden administration has claw...
28361,"Late last year, the White House persuaded Ariz...",5.278596,-2.919935,-0.322822,2.819602,-2.090031,-2.463582,-0.311356,WASHINGTON — The Biden administration has claw...
28362,Yet administration officials are less concerne...,-0.367214,-2.063666,0.582598,0.422793,-3.704168,-1.220427,6.007252,WASHINGTON — The Biden administration has claw...


In [17]:
idno, levels = pd.factorize(df_fil.text)
df_fil["id"] = idno
df_fil

C:\Users\Prannaya\AppData\Local\Temp/ipykernel_22956/1095200486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fil["id"] = idno


,sentence,Claim,Concluding Statement,Counterclaim,Evidence,Lead,Position,Rebuttal,text,id
0,In 2019 a wave of anti-abortion laws swept thi...,6.635360,-2.915717,-0.175175,1.890639,-2.265393,-2.164053,-0.640354,In 2019 a wave of anti-abortion laws swept thi...,0
1,But these grabbed the public’s attention in a ...,-0.422903,-1.966259,0.220908,0.146967,-3.457461,-0.901608,6.126251,In 2019 a wave of anti-abortion laws swept thi...,0
2,Georgia banned abortion after about six weeks ...,1.859675,-3.230959,-0.389729,6.427304,-1.808647,-3.455705,-0.482240,In 2019 a wave of anti-abortion laws swept thi...,0
3,"Ohio, Mississippi, Louisiana and Kentucky did ...",0.327022,-2.367975,-1.594195,7.579478,-1.537600,-3.283216,-0.565419,In 2019 a wave of anti-abortion laws swept thi...,0
4,"Alabama went the furthest, banning virtually a...",6.683917,-2.563557,-0.671641,1.514271,-2.902210,-1.887146,-0.006093,In 2019 a wave of anti-abortion laws swept thi...,0
...,...,...,...,...,...,...,...,...,...,...
28358,A spokesman for Gov.,7.302837,-2.645599,-0.514667,0.703273,-2.210048,-1.439218,-0.953753,WASHINGTON — The Biden administration has claw...,513
28359,"Jim Justice of West Virginia, a Republican, sa...",-0.826471,-2.144290,-0.996875,8.095543,-1.530808,-3.267610,-0.607475,WASHINGTON — The Biden administration has claw...,513
28361,"Late last year, the White House persuaded Ariz...",5.278596,-2.919935,-0.322822,2.819602,-2.090031,-2.463582,-0.311356,WASHINGTON — The Biden administration has claw...,513
28362,Yet administration officials are less concerne...,-0.367214,-2.063666,0.582598,0.422793,-3.704168,-1.220427,6.007252,WASHINGTON — The Biden administration has claw...,513


In [18]:
df_fil["newSentences"] = df_fil.sentence.apply(augment)
df_fil

2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 

KeyboardInterrupt: 

In [ ]:
augmented = df.explode("newSentences").rename(columns={"sentence":"original"}).rename(columns={"newSentences", "sentence"})
augmented

In [ ]:
augmented.to_csv("nytimes.csv", index=False)

In [ ]:
from google.colab import files
files.download('nytimes.csv')

In [ ]:
nytimes = pd.read_csv("./nytimes.csv")
nytimes

In [ ]:
from torch.nn.functional import softmax
